In [0]:
### 라이브러리 및 데이터 불러오기
# 필요한 라이브러리를 불러온다.
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pickle

# 데이터 전처리 방식을 지정한다.
transform = transforms.Compose([
        transforms.ToTensor(), # 데이터를 PyTorch의 Tensor 형식으로 바꾼다.
        transforms.Normalize(mean=(0.5,), std=(0.5,)) # 픽셀값 0 ~ 1 -> -1 ~ 1
])

dataPath = "/content/drive/My Drive/Colab deeplearning/MNIST"
mnist =datasets.MNIST(root='/content/drive/My Drive/Colab deeplearning/mnist', download=True, transform=transform)

# 데이터를 한번에 batch_size만큼만 가져오는 dataloader를 만든다.
dataloader = DataLoader(mnist, batch_size=60, shuffle=True)

In [0]:
import os
import imageio

if torch.cuda.is_available():
    use_gpu = True
leave_log = True
if leave_log:
    result_dir = '/content/drive/My Drive/Colab deeplearning/DCGAN_generated_images'
    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)

In [0]:
### DCGAN의 생성자
class Generator(nn.Module):
    
    # 네트워크 구조
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(in_channels=100, out_channels=28*8, 
                               kernel_size=7, stride=1, padding=0, 
                               bias=False),
            nn.BatchNorm2d(num_features=28*8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=28*8, out_channels=28*4, 
                               kernel_size=4, stride=2, padding=1, 
                               bias=False),
            nn.BatchNorm2d(num_features=28*4),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=28*4, out_channels=1, 
                               kernel_size=4, stride=2, padding=1, 
                               bias=False),
            nn.Tanh())
        
    # (batch_size x 100) 크기의 랜덤 벡터를 받아 
    # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    def forward(self, inputs):
        inputs = inputs.view(-1, 100, 1, 1)
        return self.main(inputs)

In [0]:
### DCGAN의 구분자
class Discriminator(nn.Module):
    
    # 네트워크 구조
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=28*4, 
                      kernel_size=4, stride=2, padding=1, 
                      bias=False),
            nn.BatchNorm2d(num_features=28*4),
            nn.LeakyReLU(0.2, inplace=True),
            # nn.Dropout(), # 추가(second run)
            # nn.Dropout(0.2), # 추가(second run)
            nn.Conv2d(in_channels=28*4, out_channels=28*8, 
                      kernel_size=4, stride=2, padding=1, 
                      bias=False),
            nn.BatchNorm2d(num_features=28*8),
            nn.LeakyReLU(0.2, inplace=True),
            # nn.Dropout(), # 추가(second run)
            # nn.Dropout(0.2), # 추가(second run)
            nn.Conv2d(in_channels=28*8, out_channels=1, 
                      kernel_size=7, stride=1, padding=0, 
                      bias=False),
            nn.Sigmoid())
        
    # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
    # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
    def forward(self, inputs):
        o = self.main(inputs)
        return o.view(-1, 1)

In [0]:
### 생성자와 구분자 객체 만들기
G = Generator()
D = Discriminator()

if use_gpu:
    G.cuda()
    D.cuda()

In [0]:
### 손실 함수와 최적화 기법 지정하기
# Binary Cross Entropy loss
criterion = nn.BCELoss()

# 생성자의 매개 변수를 최적화하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
# 구분자의 매개 변수를 최적화하는 Adam optimizer
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [0]:
# 학습 결과 시각화하기
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
    """Take an array of shape (n, height, width) or (n, height, width , 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""

    if type(data) == list:
	    data = np.concatenate(data)
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))

    padding = (((0, n ** 2 - data.shape[0]) ,
                (0, 1), (0, 1))  # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data , padding, mode='constant' , constant_values=1)  # pad with ones (white)

    # tilethe filters into an image
    data = data.reshape((n , n) + data.shape[1:]).transpose((0 , 2 , 1 , 3) + tuple(range(4 , data.ndim + 1)))

    data = data.reshape((n * data.shape[1] , n * data.shape[3]) + data.shape[4:])

    plt.imsave(path, data, cmap='gray')

In [0]:
if leave_log:
    train_hist = {}
    train_hist['D_losses'] = []
    train_hist['G_losses'] = []
    generated_images = []
    
z_fixed = Variable(torch.randn(5 * 5, 100), volatile=True)
if use_gpu:
    z_fixed = z_fixed.cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


In [0]:
from time import time

start = time()
### 모델 학습을 위한 반복문
# 데이터셋을 100번 돌며 학습한다.
for epoch in range(100):
    
    if leave_log:
        D_losses = []
        G_losses = []
    
    # 한번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
        # 데이터를 pytorch의 변수로 변환한다.
        real_data = Variable(real_data)

        ### 구분자 학습시키기

        # 이미지가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다.
        # 정답지에 해당하는 변수를 만든다.
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))
         
        if use_gpu:
            real_data, target_real, target_fake = real_data.cuda(), target_real.cuda(), target_fake.cuda()
            
        # 진짜 이미지를 구분자에 넣는다.
        D_result_from_real = D(real_data)
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다.
        D_loss_real = criterion(D_result_from_real, target_real)

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        if use_gpu:
            z = z.cuda()
            
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다.
        D_loss_fake = criterion(D_result_from_fake, target_fake)
        
        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake
        
        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()
        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트한다.
        D_optimizer.step()
        
        if leave_log:
            D_losses.append(D_loss.item())

        # train generator G

        ### 생성자 학습시키기
        
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        if use_gpu:
            z = z.cuda()
        
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 loss가 높아진다.
        G_loss = criterion(D_result_from_fake, target_real)
        
        # 생성자의 매개 변수의 미분값을 0으로 초기화한다.
        G.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        G_loss.backward()
        # 최적화 기법을 이용해 생성자의 매개 변수를 업데이트한다.
        G_optimizer.step()
        
        if leave_log:
            G_losses.append(G_loss.item())
    if leave_log:
        true_positive_rate = (D_result_from_real > 0.5).float().mean().item()
        true_negative_rate = (D_result_from_fake < 0.5).float().mean().item()
        base_message = ("Epoch: {epoch:<3d} D Loss: {d_loss:<8.6} G Loss: {g_loss:<8.6} "
                        "True Positive Rate: {tpr:<5.1%} True Negative Rate: {tnr:<5.1%}"
                       )
        message = base_message.format(
                    epoch=epoch,
                    d_loss=sum(D_losses)/len(D_losses),
                    g_loss=sum(G_losses)/len(G_losses),
                    tpr=true_positive_rate,
                    tnr=true_negative_rate
        )
        print(message)
    
    if leave_log:
        fake_data_fixed = G(z_fixed)
        image_path = result_dir + '/epoch{}.png'.format(epoch)
        square_plot(fake_data_fixed.view(25, 28, 28).cpu().data.numpy(), path=image_path)
        generated_images.append(image_path)
    
    if leave_log:
        train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
        train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))

torch.save(G.state_dict(), "dcgan_generator.pkl")
torch.save(D.state_dict(), "dcgan_discriminator.pkl")
with open('dcgan_train_history.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

generated_image_array = [imageio.imread(generated_image) for generated_image in generated_images]
imageio.mimsave(result_dir + '/DCGAN_generation.gif', generated_image_array, fps=5)

print("end_time : ", str(time()-start))

Epoch: 0   D Loss: 1.47231  G Loss: 0.583787 True Positive Rate: 75.0% True Negative Rate: 28.3%
Epoch: 1   D Loss: 1.47192  G Loss: 0.583538 True Positive Rate: 65.0% True Negative Rate: 26.7%
Epoch: 2   D Loss: 1.47234  G Loss: 0.584476 True Positive Rate: 75.0% True Negative Rate: 33.3%
Epoch: 3   D Loss: 1.47196  G Loss: 0.584697 True Positive Rate: 65.0% True Negative Rate: 28.3%
Epoch: 4   D Loss: 1.47297  G Loss: 0.585369 True Positive Rate: 76.7% True Negative Rate: 26.7%
Epoch: 5   D Loss: 1.47237  G Loss: 0.583723 True Positive Rate: 68.3% True Negative Rate: 20.0%
Epoch: 6   D Loss: 1.47322  G Loss: 0.582942 True Positive Rate: 76.7% True Negative Rate: 28.3%
Epoch: 7   D Loss: 1.47194  G Loss: 0.583973 True Positive Rate: 68.3% True Negative Rate: 20.0%
Epoch: 8   D Loss: 1.47143  G Loss: 0.58578  True Positive Rate: 78.3% True Negative Rate: 21.7%
Epoch: 9   D Loss: 1.47248  G Loss: 0.586454 True Positive Rate: 66.7% True Negative Rate: 13.3%
Epoch: 10  D Loss: 1.4711   G 

# first run
Discriminator의 Loss가 줄고  
Generator의 Loss가 커진다.  
그래서 매번 TP, TN의 %가 100%에 가깝다.  
D에 드랍아웃으로 학습속도를 조정한다.

~~~
Epoch: 0   D Loss: 0.657347 G Loss: 1.79985  True Positive Rate: 93.3% True Negative Rate: 100.0%
Epoch: 1   D Loss: 0.628278 G Loss: 1.87855  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 2   D Loss: 0.628829 G Loss: 1.94121  True Positive Rate: 98.3% True Negative Rate: 98.3%
Epoch: 3   D Loss: 0.669133 G Loss: 1.89132  True Positive Rate: 73.3% True Negative Rate: 61.7%
Epoch: 4   D Loss: 0.645549 G Loss: 1.97478  True Positive Rate: 98.3% True Negative Rate: 91.7%
Epoch: 5   D Loss: 0.616705 G Loss: 2.044    True Positive Rate: 71.7% True Negative Rate: 70.0%
Epoch: 6   D Loss: 0.593178 G Loss: 2.1371   True Positive Rate: 83.3% True Negative Rate: 98.3%
Epoch: 7   D Loss: 0.574316 G Loss: 2.20007  True Positive Rate: 91.7% True Negative Rate: 86.7%
Epoch: 8   D Loss: 0.561813 G Loss: 2.26673  True Positive Rate: 100.0% True Negative Rate: 96.7%
Epoch: 9   D Loss: 0.529704 G Loss: 2.35288  True Positive Rate: 98.3% True Negative Rate: 95.0%
Epoch: 10  D Loss: 0.523103 G Loss: 2.39087  True Positive Rate: 93.3% True Negative Rate: 96.7%
Epoch: 11  D Loss: 0.50858  G Loss: 2.45894  True Positive Rate: 85.0% True Negative Rate: 96.7%
Epoch: 12  D Loss: 0.50025  G Loss: 2.51635  True Positive Rate: 98.3% True Negative Rate: 100.0%
Epoch: 13  D Loss: 0.479997 G Loss: 2.57093  True Positive Rate: 98.3% True Negative Rate: 98.3%
Epoch: 14  D Loss: 0.478057 G Loss: 2.60982  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 15  D Loss: 0.464222 G Loss: 2.65745  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 16  D Loss: 0.453683 G Loss: 2.68697  True Positive Rate: 96.7% True Negative Rate: 88.3%
Epoch: 17  D Loss: 0.461006 G Loss: 2.69912  True Positive Rate: 95.0% True Negative Rate: 90.0%
Epoch: 18  D Loss: 0.454431 G Loss: 2.75516  True Positive Rate: 98.3% True Negative Rate: 100.0%
Epoch: 19  D Loss: 0.449087 G Loss: 2.75347  True Positive Rate: 61.7% True Negative Rate: 51.7%
Epoch: 20  D Loss: 0.438774 G Loss: 2.7677   True Positive Rate: 83.3% True Negative Rate: 60.0%
Epoch: 21  D Loss: 0.436161 G Loss: 2.81649  True Positive Rate: 96.7% True Negative Rate: 98.3%
Epoch: 22  D Loss: 0.425208 G Loss: 2.85393  True Positive Rate: 98.3% True Negative Rate: 100.0%
Epoch: 23  D Loss: 0.429851 G Loss: 2.86441  True Positive Rate: 98.3% True Negative Rate: 96.7%
Epoch: 24  D Loss: 0.425713 G Loss: 2.889    True Positive Rate: 98.3% True Negative Rate: 91.7%
Epoch: 25  D Loss: 0.425439 G Loss: 2.92164  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 26  D Loss: 0.412919 G Loss: 2.95466  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 27  D Loss: 0.420843 G Loss: 2.96403  True Positive Rate: 95.0% True Negative Rate: 91.7%
Epoch: 28  D Loss: 0.408439 G Loss: 2.97493  True Positive Rate: 90.0% True Negative Rate: 96.7%
Epoch: 29  D Loss: 0.424201 G Loss: 2.98135  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 30  D Loss: 0.406043 G Loss: 2.99383  True Positive Rate: 100.0% True Negative Rate: 90.0%
Epoch: 31  D Loss: 0.412645 G Loss: 2.9942   True Positive Rate: 86.7% True Negative Rate: 93.3%
Epoch: 32  D Loss: 0.400006 G Loss: 3.03751  True Positive Rate: 96.7% True Negative Rate: 96.7%
Epoch: 33  D Loss: 0.403088 G Loss: 3.0506   True Positive Rate: 86.7% True Negative Rate: 95.0%
Epoch: 34  D Loss: 0.392277 G Loss: 3.0507   True Positive Rate: 96.7% True Negative Rate: 100.0%
Epoch: 35  D Loss: 0.399662 G Loss: 3.06584  True Positive Rate: 100.0% True Negative Rate: 88.3%
Epoch: 36  D Loss: 0.403291 G Loss: 3.09976  True Positive Rate: 95.0% True Negative Rate: 88.3%
Epoch: 37  D Loss: 0.394733 G Loss: 3.08368  True Positive Rate: 100.0% True Negative Rate: 88.3%
Epoch: 38  D Loss: 0.378725 G Loss: 3.11962  True Positive Rate: 100.0% True Negative Rate: 56.7%
Epoch: 39  D Loss: 0.384481 G Loss: 3.14536  True Positive Rate: 85.0% True Negative Rate: 98.3%
Epoch: 40  D Loss: 0.390826 G Loss: 3.14186  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 41  D Loss: 0.381653 G Loss: 3.17194  True Positive Rate: 98.3% True Negative Rate: 100.0%
Epoch: 42  D Loss: 0.390252 G Loss: 3.20039  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 43  D Loss: 0.375448 G Loss: 3.15679  True Positive Rate: 78.3% True Negative Rate: 76.7%
Epoch: 44  D Loss: 0.385129 G Loss: 3.23474  True Positive Rate: 98.3% True Negative Rate: 98.3%
Epoch: 45  D Loss: 0.376172 G Loss: 3.17811  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 46  D Loss: 0.368788 G Loss: 3.23617  True Positive Rate: 96.7% True Negative Rate: 100.0%
Epoch: 47  D Loss: 0.378858 G Loss: 3.2315   True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 48  D Loss: 0.365132 G Loss: 3.26706  True Positive Rate: 96.7% True Negative Rate: 91.7%
Epoch: 49  D Loss: 0.369667 G Loss: 3.26905  True Positive Rate: 83.3% True Negative Rate: 86.7%
Epoch: 50  D Loss: 0.353463 G Loss: 3.28079  True Positive Rate: 78.3% True Negative Rate: 55.0%
Epoch: 51  D Loss: 0.359037 G Loss: 3.3021   True Positive Rate: 83.3% True Negative Rate: 43.3%
Epoch: 52  D Loss: 0.368486 G Loss: 3.30841  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 53  D Loss: 0.363294 G Loss: 3.30867  True Positive Rate: 58.3% True Negative Rate: 93.3%
Epoch: 54  D Loss: 0.351336 G Loss: 3.35199  True Positive Rate: 98.3% True Negative Rate: 98.3%
Epoch: 55  D Loss: 0.364834 G Loss: 3.32404  True Positive Rate: 95.0% True Negative Rate: 95.0%
Epoch: 56  D Loss: 0.364692 G Loss: 3.38221  True Positive Rate: 100.0% True Negative Rate: 98.3%
Epoch: 57  D Loss: 0.352099 G Loss: 3.37681  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 58  D Loss: 0.346831 G Loss: 3.43592  True Positive Rate: 93.3% True Negative Rate: 98.3%
Epoch: 59  D Loss: 0.353035 G Loss: 3.36335  True Positive Rate: 100.0% True Negative Rate: 91.7%
Epoch: 60  D Loss: 0.341188 G Loss: 3.41748  True Positive Rate: 91.7% True Negative Rate: 98.3%
Epoch: 61  D Loss: 0.346284 G Loss: 3.42907  True Positive Rate: 95.0% True Negative Rate: 100.0%
Epoch: 62  D Loss: 0.343913 G Loss: 3.44766  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 63  D Loss: 0.345587 G Loss: 3.44334  True Positive Rate: 98.3% True Negative Rate: 98.3%
Epoch: 64  D Loss: 0.342899 G Loss: 3.50151  True Positive Rate: 100.0% True Negative Rate: 96.7%
Epoch: 65  D Loss: 0.332248 G Loss: 3.49776  True Positive Rate: 98.3% True Negative Rate: 100.0%
Epoch: 66  D Loss: 0.338009 G Loss: 3.50122  True Positive Rate: 83.3% True Negative Rate: 96.7%
Epoch: 67  D Loss: 0.329293 G Loss: 3.51301  True Positive Rate: 98.3% True Negative Rate: 96.7%
Epoch: 68  D Loss: 0.34012  G Loss: 3.51794  True Positive Rate: 96.7% True Negative Rate: 96.7%
Epoch: 69  D Loss: 0.327341 G Loss: 3.554    True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 70  D Loss: 0.322663 G Loss: 3.5861   True Positive Rate: 96.7% True Negative Rate: 75.0%
Epoch: 71  D Loss: 0.322301 G Loss: 3.56865  True Positive Rate: 90.0% True Negative Rate: 86.7%
Epoch: 72  D Loss: 0.322674 G Loss: 3.57549  True Positive Rate: 96.7% True Negative Rate: 86.7%
Epoch: 73  D Loss: 0.323087 G Loss: 3.60274  True Positive Rate: 98.3% True Negative Rate: 81.7%
Epoch: 74  D Loss: 0.314697 G Loss: 3.61348  True Positive Rate: 98.3% True Negative Rate: 98.3%
Epoch: 75  D Loss: 0.328916 G Loss: 3.64344  True Positive Rate: 95.0% True Negative Rate: 88.3%
Epoch: 76  D Loss: 0.320412 G Loss: 3.61385  True Positive Rate: 96.7% True Negative Rate: 83.3%
Epoch: 77  D Loss: 0.312028 G Loss: 3.68035  True Positive Rate: 100.0% True Negative Rate: 91.7%
Epoch: 78  D Loss: 0.316576 G Loss: 3.67583  True Positive Rate: 98.3% True Negative Rate: 73.3%
Epoch: 79  D Loss: 0.318498 G Loss: 3.64365  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 80  D Loss: 0.31649  G Loss: 3.67077  True Positive Rate: 83.3% True Negative Rate: 98.3%
Epoch: 81  D Loss: 0.309031 G Loss: 3.69088  True Positive Rate: 86.7% True Negative Rate: 95.0%
Epoch: 82  D Loss: 0.321979 G Loss: 3.73459  True Positive Rate: 91.7% True Negative Rate: 100.0%
Epoch: 83  D Loss: 0.309568 G Loss: 3.71978  True Positive Rate: 95.0% True Negative Rate: 88.3%
Epoch: 84  D Loss: 0.298471 G Loss: 3.75458  True Positive Rate: 100.0% True Negative Rate: 100.0%

~~~

# second run
- 드랍아웃 0.5(default)로 주었다.
- 학습이 아예 안되었다.
- 0.2도 비슷한 결과를 보였다.  
  => 드랍아웃을 주는 것이 아니라 다른 방식으로 학습률을 처리해봐야 할 듯

~~~
Epoch: 0   D Loss: 1.44248  G Loss: 0.640108 True Positive Rate: 61.7% True Negative Rate: 50.0%
Epoch: 1   D Loss: 1.44127  G Loss: 0.639604 True Positive Rate: 70.0% True Negative Rate: 26.7%
Epoch: 2   D Loss: 1.43655  G Loss: 0.642741 True Positive Rate: 71.7% True Negative Rate: 43.3%
Epoch: 3   D Loss: 1.44233  G Loss: 0.638549 True Positive Rate: 53.3% True Negative Rate: 38.3%
Epoch: 4   D Loss: 1.43618  G Loss: 0.63935  True Positive Rate: 80.0% True Negative Rate: 33.3%
Epoch: 5   D Loss: 1.44175  G Loss: 0.640021 True Positive Rate: 66.7% True Negative Rate: 30.0%
Epoch: 6   D Loss: 1.43768  G Loss: 0.640009 True Positive Rate: 68.3% True Negative Rate: 33.3%
Epoch: 7   D Loss: 1.43885  G Loss: 0.640265 True Positive Rate: 66.7% True Negative Rate: 36.7%
Epoch: 8   D Loss: 1.44137  G Loss: 0.639427 True Positive Rate: 68.3% True Negative Rate: 33.3%
Epoch: 9   D Loss: 1.44113  G Loss: 0.640847 True Positive Rate: 75.0% True Negative Rate: 28.3%
Epoch: 10  D Loss: 1.44091  G Loss: 0.640109 True Positive Rate: 73.3% True Negative Rate: 35.0%
Epoch: 11  D Loss: 1.44208  G Loss: 0.64006  True Positive Rate: 71.7% True Negative Rate: 30.0%
Epoch: 12  D Loss: 1.4404   G Loss: 0.640521 True Positive Rate: 65.0% True Negative Rate: 28.3%
Epoch: 13  D Loss: 1.43799  G Loss: 0.639109 True Positive Rate: 73.3% True Negative Rate: 41.7%
Epoch: 14  D Loss: 1.43838  G Loss: 0.640996 True Positive Rate: 70.0% True Negative Rate: 31.7%
Epoch: 15  D Loss: 1.44016  G Loss: 0.639545 True Positive Rate: 65.0% True Negative Rate: 40.0%
Epoch: 16  D Loss: 1.4398   G Loss: 0.64108  True Positive Rate: 78.3% True Negative Rate: 25.0%
Epoch: 17  D Loss: 1.43961  G Loss: 0.639143 True Positive Rate: 66.7% True Negative Rate: 46.7%
Epoch: 18  D Loss: 1.43803  G Loss: 0.640246 True Positive Rate: 61.7% True Negative Rate: 30.0%
Epoch: 19  D Loss: 1.43961  G Loss: 0.637987 True Positive Rate: 73.3% True Negative Rate: 40.0%
Epoch: 20  D Loss: 1.4392   G Loss: 0.643208 True Positive Rate: 71.7% True Negative Rate: 40.0%
Epoch: 21  D Loss: 1.44124  G Loss: 0.641082 True Positive Rate: 68.3% True Negative Rate: 35.0%
Epoch: 22  D Loss: 1.43962  G Loss: 0.640725 True Positive Rate: 80.0% True Negative Rate: 46.7%
Epoch: 23  D Loss: 1.43779  G Loss: 0.641029 True Positive Rate: 66.7% True Negative Rate: 26.7%
Epoch: 24  D Loss: 1.43888  G Loss: 0.637652 True Positive Rate: 71.7% True Negative Rate: 43.3%
Epoch: 25  D Loss: 1.43841  G Loss: 0.639017 True Positive Rate: 66.7% True Negative Rate: 40.0%
Epoch: 26  D Loss: 1.44096  G Loss: 0.640166 True Positive Rate: 61.7% True Negative Rate: 38.3%
Epoch: 27  D Loss: 1.43724  G Loss: 0.639559 True Positive Rate: 63.3% True Negative Rate: 31.7%
Epoch: 28  D Loss: 1.44094  G Loss: 0.639214 True Positive Rate: 71.7% True Negative Rate: 35.0%
Epoch: 29  D Loss: 1.44068  G Loss: 0.639973 True Positive Rate: 71.7% True Negative Rate: 35.0%
Epoch: 30  D Loss: 1.43784  G Loss: 0.638481 True Positive Rate: 65.0% True Negative Rate: 23.3%
Epoch: 31  D Loss: 1.44315  G Loss: 0.639506 True Positive Rate: 73.3% True Negative Rate: 33.3%
Epoch: 32  D Loss: 1.43811  G Loss: 0.640148 True Positive Rate: 68.3% True Negative Rate: 31.7%
Epoch: 33  D Loss: 1.43958  G Loss: 0.638246 True Positive Rate: 68.3% True Negative Rate: 41.7%
Epoch: 34  D Loss: 1.44075  G Loss: 0.643112 True Positive Rate: 66.7% True Negative Rate: 45.0%
Epoch: 35  D Loss: 1.43988  G Loss: 0.639349 True Positive Rate: 61.7% True Negative Rate: 35.0%
Epoch: 36  D Loss: 1.4393   G Loss: 0.640675 True Positive Rate: 61.7% True Negative Rate: 36.7%
Epoch: 37  D Loss: 1.43966  G Loss: 0.640921 True Positive Rate: 63.3% True Negative Rate: 30.0%
Epoch: 38  D Loss: 1.43781  G Loss: 0.638805 True Positive Rate: 68.3% True Negative Rate: 41.7%
Epoch: 39  D Loss: 1.43893  G Loss: 0.63929  True Positive Rate: 71.7% True Negative Rate: 38.3%
Epoch: 40  D Loss: 1.43606  G Loss: 0.640158 True Positive Rate: 65.0% True Negative Rate: 45.0%
Epoch: 41  D Loss: 1.43936  G Loss: 0.638433 True Positive Rate: 63.3% True Negative Rate: 43.3%
Epoch: 42  D Loss: 1.43884  G Loss: 0.639613 True Positive Rate: 73.3% True Negative Rate: 36.7%
Epoch: 43  D Loss: 1.44109  G Loss: 0.638633 True Positive Rate: 80.0% True Negative Rate: 35.0%
Epoch: 44  D Loss: 1.43742  G Loss: 0.641116 True Positive Rate: 70.0% True Negative Rate: 30.0%
Epoch: 45  D Loss: 1.43818  G Loss: 0.640462 True Positive Rate: 75.0% True Negative Rate: 38.3%
Epoch: 46  D Loss: 1.44079  G Loss: 0.639866 True Positive Rate: 68.3% True Negative Rate: 41.7%
Epoch: 47  D Loss: 1.44017  G Loss: 0.639604 True Positive Rate: 70.0% True Negative Rate: 28.3%
Epoch: 48  D Loss: 1.4377   G Loss: 0.639668 True Positive Rate: 55.0% True Negative Rate: 38.3%
Epoch: 49  D Loss: 1.43809  G Loss: 0.639552 True Positive Rate: 68.3% True Negative Rate: 38.3%
Epoch: 50  D Loss: 1.43954  G Loss: 0.639801 True Positive Rate: 66.7% True Negative Rate: 31.7%
Epoch: 51  D Loss: 1.43892  G Loss: 0.640845 True Positive Rate: 75.0% True Negative Rate: 33.3%
Epoch: 52  D Loss: 1.44022  G Loss: 0.639498 True Positive Rate: 66.7% True Negative Rate: 28.3%
Epoch: 53  D Loss: 1.44049  G Loss: 0.640932 True Positive Rate: 66.7% True Negative Rate: 38.3%
Epoch: 54  D Loss: 1.43833  G Loss: 0.641772 True Positive Rate: 65.0% True Negative Rate: 41.7%
Epoch: 55  D Loss: 1.43863  G Loss: 0.640349 True Positive Rate: 63.3% True Negative Rate: 35.0%
Epoch: 56  D Loss: 1.43563  G Loss: 0.639481 True Positive Rate: 68.3% True Negative Rate: 35.0%
Epoch: 57  D Loss: 1.44056  G Loss: 0.638958 True Positive Rate: 70.0% True Negative Rate: 33.3%
Epoch: 58  D Loss: 1.43947  G Loss: 0.639234 True Positive Rate: 70.0% True Negative Rate: 45.0%
Epoch: 59  D Loss: 1.43956  G Loss: 0.64318  True Positive Rate: 70.0% True Negative Rate: 38.3%
Epoch: 60  D Loss: 1.44258  G Loss: 0.639798 True Positive Rate: 66.7% True Negative Rate: 33.3%
Epoch: 61  D Loss: 1.441    G Loss: 0.639793 True Positive Rate: 70.0% True Negative Rate: 38.3%
Epoch: 62  D Loss: 1.4409   G Loss: 0.63862  True Positive Rate: 66.7% True Negative Rate: 50.0%
Epoch: 63  D Loss: 1.43781  G Loss: 0.640783 True Positive Rate: 66.7% True Negative Rate: 43.3%
Epoch: 64  D Loss: 1.44064  G Loss: 0.639642 True Positive Rate: 71.7% True Negative Rate: 45.0%
Epoch: 65  D Loss: 1.44164  G Loss: 0.639178 True Positive Rate: 70.0% True Negative Rate: 46.7%
Epoch: 66  D Loss: 1.43904  G Loss: 0.639294 True Positive Rate: 73.3% True Negative Rate: 40.0%
Epoch: 67  D Loss: 1.43822  G Loss: 0.639668 True Positive Rate: 63.3% True Negative Rate: 28.3%
Epoch: 68  D Loss: 1.44     G Loss: 0.641797 True Positive Rate: 71.7% True Negative Rate: 46.7%
Epoch: 69  D Loss: 1.44111  G Loss: 0.640076 True Positive Rate: 73.3% True Negative Rate: 33.3%
Epoch: 70  D Loss: 1.4416   G Loss: 0.640998 True Positive Rate: 83.3% True Negative Rate: 38.3%
Epoch: 71  D Loss: 1.44053  G Loss: 0.639746 True Positive Rate: 73.3% True Negative Rate: 38.3%
Epoch: 72  D Loss: 1.44164  G Loss: 0.641381 True Positive Rate: 53.3% True Negative Rate: 35.0%
Epoch: 73  D Loss: 1.44103  G Loss: 0.640556 True Positive Rate: 76.7% True Negative Rate: 38.3%
Epoch: 74  D Loss: 1.44017  G Loss: 0.640875 True Positive Rate: 66.7% True Negative Rate: 38.3%
Epoch: 75  D Loss: 1.43806  G Loss: 0.638785 True Positive Rate: 68.3% True Negative Rate: 38.3%
Epoch: 76  D Loss: 1.4382   G Loss: 0.639361 True Positive Rate: 73.3% True Negative Rate: 45.0%
Epoch: 77  D Loss: 1.43864  G Loss: 0.640659 True Positive Rate: 73.3% True Negative Rate: 35.0%
Epoch: 78  D Loss: 1.44014  G Loss: 0.640482 True Positive Rate: 70.0% True Negative Rate: 48.3%
Epoch: 79  D Loss: 1.43922  G Loss: 0.639552 True Positive Rate: 71.7% True Negative Rate: 53.3%
Epoch: 80  D Loss: 1.43966  G Loss: 0.639523 True Positive Rate: 73.3% True Negative Rate: 38.3%
Epoch: 81  D Loss: 1.43943  G Loss: 0.638308 True Positive Rate: 73.3% True Negative Rate: 35.0%
Epoch: 82  D Loss: 1.43652  G Loss: 0.638584 True Positive Rate: 71.7% True Negative Rate: 41.7%
Epoch: 83  D Loss: 1.43999  G Loss: 0.64007  True Positive Rate: 63.3% True Negative Rate: 26.7%
Epoch: 84  D Loss: 1.43898  G Loss: 0.640707 True Positive Rate: 68.3% True Negative Rate: 26.7%
Epoch: 85  D Loss: 1.43903  G Loss: 0.641423 True Positive Rate: 65.0% True Negative Rate: 41.7%
Epoch: 86  D Loss: 1.4378   G Loss: 0.639736 True Positive Rate: 61.7% True Negative Rate: 38.3%
Epoch: 87  D Loss: 1.43609  G Loss: 0.640269 True Positive Rate: 65.0% True Negative Rate: 36.7%
Epoch: 88  D Loss: 1.4406   G Loss: 0.641141 True Positive Rate: 55.0% True Negative Rate: 28.3%
Epoch: 89  D Loss: 1.44014  G Loss: 0.640228 True Positive Rate: 65.0% True Negative Rate: 50.0%
Epoch: 90  D Loss: 1.4428   G Loss: 0.63887  True Positive Rate: 70.0% True Negative Rate: 33.3%
Epoch: 91  D Loss: 1.44013  G Loss: 0.639831 True Positive Rate: 66.7% True Negative Rate: 43.3%
Epoch: 92  D Loss: 1.44023  G Loss: 0.640245 True Positive Rate: 68.3% True Negative Rate: 36.7%
Epoch: 93  D Loss: 1.44158  G Loss: 0.639834 True Positive Rate: 61.7% True Negative Rate: 38.3%
Epoch: 94  D Loss: 1.43831  G Loss: 0.638706 True Positive Rate: 63.3% True Negative Rate: 40.0%
Epoch: 95  D Loss: 1.4394   G Loss: 0.640516 True Positive Rate: 68.3% True Negative Rate: 45.0%
Epoch: 96  D Loss: 1.4434   G Loss: 0.640182 True Positive Rate: 68.3% True Negative Rate: 26.7%
Epoch: 97  D Loss: 1.44238  G Loss: 0.639758 True Positive Rate: 68.3% True Negative Rate: 26.7%
Epoch: 98  D Loss: 1.43719  G Loss: 0.638687 True Positive Rate: 71.7% True Negative Rate: 36.7%
Epoch: 99  D Loss: 1.4385   G Loss: 0.641829 True Positive Rate: 71.7% True Negative Rate: 30.0%
/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
end_time :  3316.097769498825
~~~